In [68]:
library(tidyverse)

# Custom package
library(rutils)

In [69]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
proj <- c("CESC", "UCS", "UCEC")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [70]:
matrisome_df <- load_matrisome_df(matrisome_path)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)




In [71]:
hub_con_thresh <- 0.0
dset_idx <- 3

In [72]:
demg_df <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_dge_m_gene_list.txt")) %>%
    as_tibble() %>%
    dplyr::rename(geneID = value)
dge_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv"))
nrow(demg_df)
nrow(dge_df)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double(),
  qval = col_double()
)




[1] 618

[1] 18135

In [73]:
demg_details_df <- dge_df %>%
    dplyr::inner_join(demg_df, by = "geneID") %>%
    dplyr::inner_join(matrisome_df, by = c("geneID" = "gene_symbol")) %>%
    dplyr::select(-c(lfcSE, stat, pvalue, padj, synonyms, hgnc_ids_links, uniprot_ids, refseq_ids, orthology, notes))

In [74]:
write_tsv(demg_details_df, paste0(dirs$analysis_dir, "/one_off/", unified_dsets[dset_idx], "_demg_details.tsv"))